### udemy

In [40]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# PrefetchDataset
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
# PrefetchDataset
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset['test']

# EagerTensor
# なぜtf.castを通しているのかは現段階では不明、おそらくtf内で計算しやすいオブジェクトにしているのではなかろうか
num_valid_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)


# Feature Scaling 0~255 => 0~1 にスケーリングする
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


# MapDataset
scaled_train_and_valid_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# train/validにsplit
BUFFER_SIZE = 10000 # メモリ対策
shuffled_train_and_valid_data = scaled_train_and_valid_data.shuffle(BUFFER_SIZE)
valid_data = shuffled_train_and_valid_data.take(num_valid_samples)
train_data = shuffled_train_and_valid_data.skip(num_valid_samples)

# 負荷対策
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
valid_data = valid_data.batch(num_valid_samples)
test_data = test_data.batch(num_test_samples)

valid_inputs, valid_targets = next(iter(valid_data))

In [41]:
#
# モデルの作成
# 隠れ層: 2層
# 合計: 4層
#
input_size = 784 # 28 * 28 * 1
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # 1列に変換
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

In [42]:
#
# 最適化アルゴリズムと損失関数の決定
#

# sparse_categorical_crossentropy: ワンホットエンコーディングで使うらしい
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [46]:
#
# 訓練
#
NUM_EPOCHS = 5 # 繰り返し回数
VALIDATION_STEPS = num_valid_samples # バッチサイズ
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(valid_inputs, valid_targets), validation_steps=VALIDATION_STEPS, verbose=2)

Epoch 1/5
540/540 - 4s - loss: 0.0137 - accuracy: 0.9959 - val_loss: 0.0167 - val_accuracy: 0.9953
Epoch 2/5
540/540 - 4s - loss: 0.0126 - accuracy: 0.9959 - val_loss: 0.0197 - val_accuracy: 0.9933
Epoch 3/5
540/540 - 5s - loss: 0.0117 - accuracy: 0.9964 - val_loss: 0.0205 - val_accuracy: 0.9922
Epoch 4/5
540/540 - 4s - loss: 0.0122 - accuracy: 0.9961 - val_loss: 0.0211 - val_accuracy: 0.9940
Epoch 5/5
540/540 - 4s - loss: 0.0137 - accuracy: 0.9958 - val_loss: 0.0331 - val_accuracy: 0.9888


In [49]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 224ms/step - loss: 0.1390 - accuracy: 0.9726
Test loss: 0.14. Test accuracy: 97.26%
